In [18]:
#from __future__ import print_function, division
import os
import torch
import pandas as pd
#from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
#from torchvision import transforms, utils

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils, generic_utils


# Ignore warnings
#import warnings
#warnings.filterwarnings("ignore")

#plt.ion()   # interactive mode

Using TensorFlow backend.


In [7]:
# import configurations (file paths, etc.)
import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
    
configFile = '../cluster/data/medinfmk/ddi/config/config.yml'

with open(configFile, 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=Loader)
    
#print(cfg)

In [8]:
pathInput = cfg['filePaths']['dirRaw']
pathOutput = cfg['filePaths']['dirProcessed']
# path to store python binary files (pickles)
# in order not to recalculate them every time
pathPickles = cfg['filePaths']['dirProcessedFiles']['dirPickles']
datasetDirs = cfg['filePaths']['dirRawDatasets']
DS1_path = str(datasetDirs[0])

In [9]:
#landmarks_frame = pd.read_csv('data/faces/face_landmarks.csv')

# NDD Methods

In [10]:
def prepare_data(input_fea, input_lab, seperate=False):
    offside_sim_path = input_fea
    drug_interaction_matrix_path = input_lab
    drug_fea = np.loadtxt(offside_sim_path,dtype=float,delimiter=",")
    interaction = np.loadtxt(drug_interaction_matrix_path,dtype=int,delimiter=",")
    train = []
    label = []
    tmp_fea=[]
    drug_fea_tmp = []
    for i in range(0, interaction.shape[0]):
        for j in range(0, interaction.shape[1]):
            label.append(interaction[i,j])
            drug_fea_tmp = list(drug_fea[i])
            if seperate:
        
                 tmp_fea = (drug_fea_tmp,drug_fea_tmp)

            else:
                 tmp_fea = drug_fea_tmp + drug_fea_tmp
            train.append(tmp_fea)

    return np.array(train), label

In [11]:
def transfer_array_format(data):
    formated_matrix1 = []
    formated_matrix2 = []
    for val in data:
        formated_matrix1.append(val[0])
        formated_matrix2.append(val[1])
    return np.array(formated_matrix1), np.array(formated_matrix2)

In [12]:
def preprocess_labels(labels, encoder=None, categorical=True):
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
        y = encoder.transform(labels).astype(np.int32)
    if categorical:
        y = np_utils.to_categorical(y)
        print(y)
    return y, encoder

In [13]:
def preprocess_names(labels, encoder=None, categorical=True):
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
    if categorical:
        labels = np_utils.to_categorical(labels)
    return labels, encoder

In [14]:
#def NDD(input_dim): 
#    model = Sequential()
#    model.add(Dense(input_dim=input_dim, output_dim=400,init='glorot_normal'))
#    model.add(Activation('relu'))
#    model.add(Dropout(0.5))
#    model.add(Dense(input_dim=400, output_dim=300,init='glorot_normal'))
#    model.add(Activation('relu'))
#    model.add(Dropout(0.5))
#    model.add(Dense(input_dim=300, output_dim=2,init='glorot_normal'))
#    model.add(Activation('sigmoid'))
#    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#    model.compile(loss='binary_crossentropy', optimizer=sgd)                  
#    return model

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class NDD(torch.nn.Module):
    def __init__(self, D_in, H1, H2, D_out, drop):
        super(NDD, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(D_in, H1) # Fully Connected
        self.fc2 = nn.Linear(H1, H2)
        self.fc3 = nn.Linear(H2, D_out)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = torch.sigmoid(self.fc3(x))
        return x

# Training

### Prepare Data

In [49]:
# input_fea = pathInput+DS1_path+"/offsideeffect_Jacarrd_sim.csv"
# input_lab = pathInput+DS1_path+"/drug_drug_matrix.csv"
# X, labels = prepare_data(input_fea, input_lab, seperate = True)

# X_data1, X_data2 = transfer_array_format(X)
# y, encoder = preprocess_labels(labels)
# X = np.concatenate((X_data1, X_data2), axis = 1)
# model_input_dim = X.shape[1]

In [50]:
# Params

# Model
D_in, H1, H2, D_out, drop = model_input_dim, 400, 300, 2, 0.5
# Training
batch_size, epochs = 100, 20
print_iter = int(epochs / 10)
# SGD
learning_rate, momentum, weight_decay, nesterov = 0.01, 0.9, 1e-6, True

# Construct our model by instantiating the class defined above
model = NDD(D_in, H1, H2, D_out, drop)
criterion = torch.nn.BCELoss() # binary crossentropy
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=learning_rate, 
                            momentum=momentum, 
                            weight_decay=weight_decay, 
                            nesterov=nesterov)

In [51]:
# Create random Tensors to hold inputs and outputs
x = torch.randn(batch_size, D_in)
y = torch.randn(batch_size, D_out)

for e in range(epochs):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if e % print_iter == (print_iter-1):
        print(e, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

1 0.5951357483863831
3 0.5329524874687195
5 0.42416104674339294
7 0.31613779067993164
9 0.16827653348445892
11 0.07982834428548813
13 -0.08929590880870819
15 -0.20616117119789124
17 -0.4190223813056946
19 -0.6757856011390686
